# Project: Building fast Queries on a CSV

## Table of Contents
* Introduction
* Data Wrangling
* Creating class Inventory
* Query 1: Finding a Laptop from the Id
* Query 2: Two laptop promotion
* Query 3: Finding Laptops Within a Budget
* Conclusion

# Introduction
> In this project, i will use the laptop price dataset which is about laptops configuration with prices containing 1303 laptop data as an inventory.
>
> The goal of this project is to create a class that represents our inventory. The methods in that class will implement the queries that i want to answer about our inventory. In other words, We will imagine that we own an online laptop store and want to build a way to answer a few different business questions about our inventory. We will also preprocess that data to make those queries run faster.
>
> Here are some queries that we will want to answer:
> 1. Given a laptop id, find the corresponding data.
> 2. Given an amount of money, find whether there are two laptops whose total > price is that given amount.
> 3. Identify all laptops whose price falls within a given budget.

# Data Wrangling

## Gather
let's start by importing some of the libraries we need

In [1]:
from csv import reader

## Loading Data

In [2]:
# Use the python built in function open() to open the hacker news data
opened_file = open('laptops.csv')

# Use csv.reader() to parse the data from the opened file
read_file = reader(opened_file)

# Use list() to convert the read file into a list of lists format
laptops = list(read_file)

#Close the opened file
opened_file.close()

# Assign first row to header
header = laptops[0]

# Remove the first row of the data, which contains the column names
rows = laptops[1:]

## Assess Data

In [3]:
# Print number of rows and column
print('Number of rows:', len(rows))
print('Number of columns:', len(rows[0]))

Number of rows: 1303
Number of columns: 13


In [4]:
# print column names
header

['Id',
 'Company',
 'Product',
 'TypeName',
 'Inches',
 'ScreenResolution',
 'Cpu',
 'Ram',
 'Memory',
 'Gpu',
 'OpSys',
 'Weight',
 'Price']

Here is a brief description of the column names:

* ID: A unique identifier for the laptop.
* Company: The name of the company that produces the laptop.
* Product: The name of the laptop.
* TypeName: The type of laptop.
* Inches: The size of the screen in inches.
* ScreenResolution: The resolution of the screen.
* CPU: The laptop CPU.
* RAM: The amount of RAM in the laptop.
* Memory: The size of the hard drive.
* GPU: The graphics card name.
* OpSys: The name of the operating system.
* Weight: The laptop weight.
* Price: The price of the laptop.

### Let's explore the data by printing the first 5 rows

In [5]:
rows[0:5]

[['6571244',
  'Apple',
  'MacBook Pro',
  'Ultrabook',
  '13.3',
  'IPS Panel Retina Display 2560x1600',
  'Intel Core i5 2.3GHz',
  '8GB',
  '128GB SSD',
  'Intel Iris Plus Graphics 640',
  'macOS',
  '1.37kg',
  '1339'],
 ['7287764',
  'Apple',
  'Macbook Air',
  'Ultrabook',
  '13.3',
  '1440x900',
  'Intel Core i5 1.8GHz',
  '8GB',
  '128GB Flash Storage',
  'Intel HD Graphics 6000',
  'macOS',
  '1.34kg',
  '898'],
 ['3362737',
  'HP',
  '250 G6',
  'Notebook',
  '15.6',
  'Full HD 1920x1080',
  'Intel Core i5 7200U 2.5GHz',
  '8GB',
  '256GB SSD',
  'Intel HD Graphics 620',
  'No OS',
  '1.86kg',
  '575'],
 ['9722156',
  'Apple',
  'MacBook Pro',
  'Ultrabook',
  '15.4',
  'IPS Panel Retina Display 2880x1800',
  'Intel Core i7 2.7GHz',
  '16GB',
  '512GB SSD',
  'AMD Radeon Pro 455',
  'macOS',
  '1.83kg',
  '2537'],
 ['8550527',
  'Apple',
  'MacBook Pro',
  'Ultrabook',
  '13.3',
  'IPS Panel Retina Display 2560x1600',
  'Intel Core i5 3.1GHz',
  '8GB',
  '256GB SSD',
  'Intel

# Creating our class - Inventory

In [6]:
# Create a class named Inventory
class Inventory():
    def __init__(self, csv_filename):
        file = open(csv_filename)
        from csv import reader
        read_file = reader(file)
        rows = list(read_file) #saving file as list of lists
        self.header = rows[0] #first row
        self.rows = rows[1:] #removes the header
        for row in self.rows: #converts the price of each row to an integer
            row[-1] = int(row[-1])

#### Testing the class by creating an instance of Inventory using laptops.csv

In [7]:
inventory = Inventory('laptops.csv')
print(inventory.header)  
print(len(inventory.rows))

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']
1303


# Query 1: Finding a Laptop from the Id

The first thing that we will implement is a way to look up a laptop from a given identifier. In this way, when a customer comes to our store with a purchase slip, we can quickly identify the laptop to which it corresponds.

For this, we will write a function named get_laptop_from_id(). This function will take as argument the identifier of the laptop and return the full row of the laptop with that id.

In [8]:
# Modify class Inventory
class Inventory():
    def __init__(self, csv_filename):
        file = open(csv_filename)
        from csv import reader
        read_file = reader(file)
        rows = list(read_file) #saving file as list of lists
        self.header = rows[0] #first row
        self.rows = rows[1:] #removes the header
        for row in self.rows: #converts the price of each row to an integer
            row[-1] = int(row[-1])
            
    def get_laptop_from_id(self, laptop_id):   #step 1
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None

## Testing query 1

In [9]:
inventory = Inventory('laptops.csv')
print(inventory.get_laptop_from_id('3362737'))  
print(inventory.get_laptop_from_id('3362736'))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


## Improving query 1 by preprocessing the data

The idea is preprocess the data into a dictionary where the keys are the IDs and the values the rows. Then, we will use that dictionary in the get_laptop_from_id() method. We can do this by:

1. Preprocess the data and create the dictionary in the __init__() method.
2. Re-implement the get_laptop_from_id() method. We will do it as a new method so that we can compare the two.

N.B: Using a set, we can check in constant time whether a given id exists. However, we don't just want to know if it exists, we also want to retrieve the remaining row information. Therefore, we will use a dictionary instead of a set. Dictionaries have the same fast lookup properties that sets have, but allow us to associate values to the keys.

In [10]:
# Modify class Inventory
class Inventory():
    def __init__(self, csv_filename):
        file = open(csv_filename)
        from csv import reader
        read_file = reader(file)
        rows = list(read_file) #saving file as list of lists
        self.header = rows[0] #first row
        self.rows = rows[1:] #removes the header
        for row in self.rows: #converts the price of each row to an integer
            row[-1] = int(row[-1])
        
        self.id_to_row = {}        #step 1
        for row in self.rows:
            self.id_to_row[row[0]] = row
            
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None        
    
    def get_laptop_from_id_fast(self, laptop_id):     #step 2
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None

## Testing modified query 1

In [11]:
inventory = Inventory('laptops.csv')
print(inventory.get_laptop_from_id_fast('3362737'))  
print(inventory.get_laptop_from_id_fast('3362736'))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


## Comparing the performance of Query 1 and Modified Query 1

The idea is to generate random IDs using the random module. Then, use both methods to lookup these same IDs. We will use the time module to measure the execution time of each lookup and, for each method, add all times together.

In [12]:
import time
import random

In [13]:
ids = [str(random.randint(1000000, 9999999)) for _ in range(10000)] #Generate a list named ids with 10,000 random values between 1000000 and 9999999 (this is the id range

In [14]:
ids[59]

'2765297'

In [15]:
inventory = Inventory('laptops.csv')
total_time_no_dict = 0 #This variable will aggregate the times of calling get_laptop_from_id()
for identifier in ids:
    start = time.time()        
    inventory.get_laptop_from_id(identifier)
    end = time.time()     
    runtime1 = end - start
    total_time_no_dict += runtime1

In [16]:
inventory = Inventory('laptops.csv')
total_time_dict = 0 #This variable will aggregate the times of calling get_laptop_from_id_fast()
for identifier in ids:
    start = time.time()        
    inventory.get_laptop_from_id_fast(identifier)
    end = time.time()     
    runtime2 = end - start
    total_time_dict += runtime2

In [17]:
total_time_no_dict

1.3493232727050781

In [18]:
total_time_dict

0.005632638931274414

In [19]:
total_time_no_dict / total_time_dict

239.55437037037038

## Analysis

We can see a significant improve in performance. We see that the new method is about 239 times faster for this input size.

# Query 2: Two laptop promotion

The idea is to write a function that, given a dollar amount, checks whether it is possible to spend precisely that amount by purchasing up to two laptops.

Imagine you issue a gift card to a customer.You don't want to make a customer feel cheated, so whenever you issue a gift card, you want to make sure that there is at least one way to spend it in full. In other words, before issuing a gift card for D dollars, you want to make sure that either there is a laptop that costs exactly D dollars or two laptops whose costs add up to precisely D dollars.

In [20]:
# Modify class Inventory
class Inventory():
    def __init__(self, csv_filename):
        file = open(csv_filename)
        from csv import reader
        read_file = reader(file)
        rows = list(read_file) #saving file as list of lists
        self.header = rows[0] #first row
        self.rows = rows[1:] #removes the header
        for row in self.rows: #converts the price of each row to an integer
            row[-1] = int(row[-1])
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row
            
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None        
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):   #step1
        for row in self.rows:
            if row[-1] == dollars:
                return True
        for row1 in self.rows:
            for row2 in self.rows:
                if row1[-1] + row2[-1] == dollars:
                    return True
        return False
            

## Testing Query 2

In [21]:
inventory = Inventory('laptops.csv')
print(inventory.check_promotion_dollars(1000))  
print(inventory.check_promotion_dollars(442))

True
False


## Optimizing laptop promotion(Query 2) by preprocessing the data

N.B: Since we only care about whether or not there is a solution, we can store all laptops prices in a set when we initialize the inventory. Then we can check in constant time whether there is a laptop with a given price.

In [22]:
# Modify class Inventory
class Inventory():
    def __init__(self, csv_filename):
        file = open(csv_filename)
        from csv import reader
        read_file = reader(file)
        rows = list(read_file) #saving file as list of lists
        self.header = rows[0] #first row
        self.rows = rows[1:] #removes the header
        for row in self.rows: #converts the price of each row to an integer
            row[-1] = int(row[-1])
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row
        self.prices = set()    #step 1
        for row in self.rows:
            self.prices.add(row[-1])
            
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None        
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True
        for row1 in self.rows:
            for row2 in self.rows:
                if row1[-1] + row2[-1] == dollars:
                    return True
        return False
    
    def check_promotion_dollars_fast(self, dollars):  #step 2
        if self.prices == dollars:
                return True
        for price1 in self.prices:
            for price2 in self.prices:
                if price1 + price2 == dollars:
                    return True
        return False

## Testing modified query 2

In [23]:
inventory = Inventory('laptops.csv')
print(inventory.check_promotion_dollars_fast(1000))  
print(inventory.check_promotion_dollars_fast(442))

True
False


## Comparing the performance of Query 2 and Modified Query 2

In [24]:
prices =  [random.randint(100, 500) for _ in range(100)] #Generate a list named prices with 100 random values between 100 and 5,000

In [25]:
inventory = Inventory('laptops.csv')
total_time_no_set = 0 #This variable will aggregate the times of calling check_promotion-dollars()
for price in prices:
    start = time.time()        
    inventory.check_promotion_dollars(price)
    end = time.time()     
    runtime1 = end - start
    total_time_no_set += runtime1
    
total_time_set = 0 #This variable will aggregate the times of calling check_promotion-dollars_fast()
for price in prices:
    start = time.time()        
    inventory.check_promotion_dollars_fast(price)
    end = time.time()     
    runtime2 = end - start
    total_time_set += runtime2
    
print(total_time_no_set)
print(total_time_set)

17.686903715133667
4.392803430557251


In [26]:
total_time_no_set / total_time_set

4.026336255362555

## Analysis

We can see an improvement in performance by using the modified method

# Query 3: Finding Laptops Within a Budget

We want to write a method that efficiently answers the query: Given a budget of D dollars, find all laptops whose price it at most D.

In [27]:
# Modify class Inventory
def row_price(row):
    return row[-1]

class Inventory():
    def __init__(self, csv_filename):
        file = open(csv_filename)
        from csv import reader
        read_file = reader(file)
        rows = list(read_file) #saving file as list of lists
        self.header = rows[0] #first row
        self.rows = rows[1:] #removes the header
        for row in self.rows: #converts the price of each row to an integer
            row[-1] = int(row[-1])
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row
        self.prices = set()
        for row in self.rows:
            self.prices.add(row[-1])
        self.rows_by_price = sorted(self.rows, key = row_price)
            
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None        
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True
        for row1 in self.rows:
            for row2 in self.rows:
                if row1[-1] + row2[-1] == dollars:
                    return True
        return False
    
    def check_promotion_dollars_fast(self, dollars):
        if self.prices == dollars:
                return True
        for price1 in self.prices:
            for price2 in self.prices:
                if price1 + price2 == dollars:
                    return True
        return False
    
    def find_laptop_with_price(self, target_price):
        range_start = 0                                   
        range_end = len(self.rows_by_price) - 1
        while range_start < range_end:
            range_middle = (range_end + range_start) // 2  
            price = self.rows_by_price[range_middle][-1]
            if price == target_price:                            
                return range_middle                        
            elif price < target_price:                           
                range_start = range_middle + 1             
            else:                                          
                range_end = range_middle - 1 
        price = self.rows_by_price[range_start][-1]
        if price != target_price:                  
            return -1                                      
        return range_start
   
    
    def find_first_laptop_more_expensive(self, target_price):
        range_start = 0                                   
        range_end = len(self.rows_by_price) - 1                   
        while range_start < range_end:
            range_middle = (range_end + range_start) // 2  
            price = self.rows_by_price[range_middle][-1]
            if price > target_price:
                range_end = range_middle
            else:
                range_start = range_middle + 1
        if self.rows_by_price[range_start][-1] <= target_price:                  
            return -1                                   
        return range_start

In [28]:
inventory = Inventory('laptops.csv')                                
print(inventory.find_first_laptop_more_expensive(1000))  
print(inventory.find_first_laptop_more_expensive(10000))

683
-1


# Conclusion

The important takeaway is that there are multiple ways of writing an algorithm for the same problem. Each solution will have a different time and space complexity trade-off. Which algorithm is better often depends on the exact setting over which it will be used. When designing algorithms, it is important to have the usage scenario in mind to make the best choices.

Also,binary search is a very efficient way to lookup a value in a sorted list and it can be used to save a lot of time when we need to perform significant number of lookups. 